# lighteval es tu biblioteca de evaluación de IA

Este cuaderno explora cómo puedes usar lighteval para evaluar y comparar modelos de lenguaje grandes (LLMs).

`lighteval` ha estado disponible por un tiempo y es una gran herramienta para obtener puntuaciones de evaluación en los principales "benchmarks". Recientemente ha sido refactorizado para poder usarse como una biblioteca en Python, lo que la hace excelente para comparar modelos en diferentes "benchmarks".

Así que exploremos algunas puntuaciones de evaluación.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Ejercicio: Evalúa tu propio modelo</h2>
    <p>Ahora que has visto cómo evaluar modelos en dominios específicos, intenta evaluar un modelo en un dominio que te interese.</p>
    <p><b>Niveles de dificultad</b></p>
    <p>🐢 Usa las tareas existentes del dominio médico pero evalúa un modelo diferente del repositorio de Hugging Face</p>
    <p>🐕 Crea una nueva evaluación de dominio seleccionando diferentes tareas de MMLU (por ejemplo, ciencias de la computación, matemáticas, física)</p>
    <p>🦁 Crea una tarea de evaluación personalizada utilizando el marco de tareas de LightEval y evalúa modelos en tu dominio específico</p>
</div>

## Instalar dependencias

In [ ]:
!pip install -qqq -U "torch<2.5" "torchvision<2.5" "torchaudio<2.5" --index-url https://download.pytorch.org/whl/cu121
!pip list | grep torch

In [ ]:
!pip install -qqq git+https://github.com/huggingface/lighteval.git tiktoken


## Configurar la evaluación con `lighteval`

Necesitamos configurar el entorno de evaluación y el pipeline. Muchas de estas configuraciones estarán desactivadas porque trabajaremos directamente en el notebook, pero también podríamos usar `push_to_hub` o `push_to_tensorboard`.

### `push_to_hub`

Esto es útil si estamos evaluando un modelo y queremos guardar su evaluación junto con los pesos y la configuración en el repositorio de Hugging Face.

### `push_to_tensorboard`

Esto sería útil si estamos construyendo una herramienta o script de evaluación y queremos visualizar los resultados en tensorboard.


In [ ]:
import lighteval
import os
from datetime import timedelta
from transformers import AutoModelForCausalLM

from lighteval.logging.evaluation_tracker import EvaluationTracker
from lighteval.logging.hierarchical_logger import hlog_warn, htrack
from lighteval.models.model_config import create_model_config
from lighteval.pipeline import EnvConfig, ParallelismManager, Pipeline, PipelineParameters

TOKEN = os.getenv("HF_TOKEN")

In [ ]:
env_config = EnvConfig(token=TOKEN, cache_dir="~/tmp")

evaluation_tracker = EvaluationTracker(
    output_dir="~/tmp",
    save_details=False,
    push_to_hub=False,
    push_to_tensorboard=False,
    public=False,
    hub_results_org=False,
)

pipeline_params = PipelineParameters(
    launcher_type=ParallelismManager.ACCELERATE,
    env_config=env_config,
    job_id=1,
    override_batch_size=1,
    num_fewshot_seeds=0,
    max_samples=10,
    use_chat_template=False,
)

# Comparar modelos con `lighteval`

Vamos a comparar dos modelos pequeños en un dominio específico. Usaremos `Qwen2.5-0.5B` y `SmolLM2-360M-Instruct`, y los evaluaremos en un dominio médico.

Podemos crear una evaluación de dominio a partir de un subconjunto de evaluaciones de MMLU definiendo las tareas de evaluación. En lighteval, las tareas se describen como cadenas.

`{suite}|{task}:{subtask}|{num_few_shot}|{0 o 1 para reducir num_few_shot si el "prompt" es demasiado largo}`

Por lo tanto, pasaremos nuestra lista de tareas relacionadas con medicina de esta forma:

```
"leaderboard|mmlu:anatomy|5|0,leaderboard|mmlu:professional_medicine|5|0,leaderboard|mmlu:high_school_biology|5|0,leaderboard|mmlu:high_school_chemistry|5|0"
```

Que se traduce a:

| Suite | Tarea | Ejemplos Fewshot | Limitar Fewshots |
|---|---|---|---|
| leaderboard | mmlu:anatomy | 5 | Falso |
| leaderboard | mmlu:professional_medicine | 5 | Falso |
| leaderboard | mmlu:high_school_biology | 5 | Falso |
| leaderboard | mmlu:high_school_chemistry | 5 | Falso |

Para obtener una lista completa de las tareas soportadas por lighteval, consulta esta página en [la documentación](https://github.com/huggingface/lighteval/wiki/Available-Tasks).



In [ ]:
domain_tasks = "leaderboard|mmlu:anatomy|5|0,leaderboard|mmlu:professional_medicine|5|0,leaderboard|mmlu:high_school_biology|5|0,leaderboard|mmlu:high_school_chemistry|5|0"

# Evaluación Qwen2.5 0.5B

In [ ]:
qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B")

pipeline = Pipeline(
    tasks=domain_tasks,
    pipeline_parameters=pipeline_params,
    evaluation_tracker=evaluation_tracker,
    model=qwen_model
)

pipeline.evaluate()

qwen_results = pipeline.get_results()

# Evaluación SmolLM 360M

In [ ]:
smol_model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")

pipeline = Pipeline(
    tasks=domain_tasks,
    pipeline_parameters=pipeline_params,
    evaluation_tracker=evaluation_tracker,
    model=smol_model
)

pipeline.evaluate()

In [ ]:
smol_results = pipeline.get_results()

In [ ]:
pipeline.show_results()

# Visualiza los Resultados

Ahora que tenemos los resultados de los dos modelos, podemos visualizarlos lado a lado. Mantendremos la visualización simple, pero con esta estructura de datos podrías representar las puntuaciones de muchas maneras.

In [ ]:
import pandas as pd

df = pd.DataFrame.from_records(smol_results["results"]).T["acc"].rename("SmolLM2-360M-Instruct")
_df = pd.DataFrame.from_records(qwen_results["results"]).T["acc"].rename("Qwen2-0.5B-DPO")
df = pd.concat([df, _df], axis=1)
df.plot(kind="barh")

# 💐 ¡Eso es todo!

Tienes un cuaderno práctico para visualizar las evaluaciones de modelos. Puedes usarlo para:

- seleccionar el modelo adecuado para tu caso de uso
- evaluar "checkpoints" durante el entrenamiento
- compartir puntuaciones de modelos

🏃 Próximos pasos

- Si quieres profundizar en tus resultados de evaluación, consulta este [cuaderno](https://github.com/huggingface/evaluation-guidebook/blob/main/contents/examples/comparing_task_formulations.ipynb).